# KPMG TEST NOTEBOOK

Selenium is a tool initially created to automate tests on websites. It is therefore very useful when information is accessible by clicking on links. A button for example is an element from which it is very difficult to obtain the link. BeautifulSoup then becomes limited.
In this case, use Selenium.

In [2]:

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from random import randrange
import pandas as pd
import html2text

In [3]:
#!python -m spacy download en_core_news_md

import pandas as pd
import spacy
import re

# Load the model
nlp = spacy.load("nl_core_news_md")
stopwords = nlp.Defaults.stop_words

def dict_sort(dict_uniqwords:dict ) -> dict:
    from operator import itemgetter
    srt=sorted(dict_uniqwords.items(), key=itemgetter(1),reverse=True)
    return srt

def getsomestats(dict_uniqwords:dict ,txt_original:str, topcnt:int = 15):
    #---NOT USED FOR NOW

    #print("@@@@@@@@@@@@@@@@@@@@",dict_uniqwords)
    # total number of unique words in doc
    print("unique word count :", len(dict_uniqwords))

    # total number of words in doc
    wordcnt=0
    for s in dict_uniqwords: wordcnt+=dict_uniqwords[s]
    print("total wordcount: ",wordcnt)
    
    #sorted words count
    ###from operator import itemgetter
    ###srt=sorted(dict_uniqwords.items(), key=itemgetter(1),reverse=True)
    srt = dict_sort(dict_uniqwords)

    res=[]
    for i in range(topcnt):
        try:
            res.append(srt[i][0])
            print(srt[i])
        except:
            break
    #print original doc
    #print(txt_original)
    return res

def nlp_cleanandlemmatize(txtdoc: str, list_wordstoskip:str = ''):
    LANG='nl'
    # Load the model
    #nlp = spacy.load("nl_core_news_md")
    #stopwords = nlp.Defaults.stop_words

    # words to discard
    months={'nl':['januari','februari','maart','april','mei','juni','augustus','september','oktober','november','december'],
            'fr':['janvier','fevrier','mars','avril','mai','juin','juillet','aout','septembre','octobre','decembre']}
    days={'nl':['maandag','dinsdag','woensdag','donderdag','vrijdag','zaterdag','zondag'],
            'fr':['lundi','mardi','mercredi','jeudi','vendredi','samedi','dimanche']}
    # 
    nlp.max_length=10000000
    nlp_doc=nlp(txtdoc)
    list_allwordslemmatized=[]
    dict_uniqwords={}
    list_tokens=[]
    #sequential filter
    for token in nlp_doc:
        lemma_lower=token.lemma_.lower()
        if token in stopwords:
            continue
        if (token.is_punct or token.is_space or token.is_stop):
            continue
        if token.text.isdecimal():
            continue
        if True in [char.isdigit() for char in token.text]:
            continue
        if token.text[-1] == '.':
            continue
        if len(token.text) <= 2:
            continue
        if lemma_lower in months[LANG]:
            continue
        if lemma_lower in days[LANG]:
            continue
        if lemma_lower in list_wordstoskip:
            continue
        #pass only nouns
        #print(token.pos_,token.pos)
        if token.pos_ != 'NOUN':
            continue
        #create dict of unique words with count
        if lemma_lower not in dict_uniqwords: 
            dict_uniqwords[lemma_lower]=1
            #save tokens for vector comparison
            list_tokens.append(token)
        else:
            dict_uniqwords[lemma_lower]+=1
        ##save tokens for vector comparison
        ##list_tokens.append(token)

        
        #saving the words (lemma equates number   -   lemma_ equates word)
        ###list_allwordslemmatized.append([token.lemma,lemma_lower,token.text])
    return dict_uniqwords, list_tokens
    #end of filter-----------------------------------------------

def concat_docs(docs:list[str] ,nmbr:int = 0) -> str:
    #concat nmbr dcocuments from doc return txtdoc
    #nmbr=10 #test anz docus
    txtdoc=''
    #for txt in df['cleantextnl'].values:
    for txt in docs:
        txtdoc+=str(txt)
        if nmbr == 1:
            break
        nmbr-=1
    return txtdoc

#read all docs
df = pd.read_pickle("../data/Staatsblad_nl_fr.pkl") 




In [4]:
#prepare testdata dict_uniqwords, list_sorted_uniqwords, list_tokens

numberofdocumenttoscan=10

for n in range(numberofdocumenttoscan):
    txtdoc = df['cleantextnl'].values[n]

    list_wordstoskip=['artikel','document'] #add here words to discard

    dict_uniqwords,list_tokens=nlp_cleanandlemmatize(txtdoc,list_wordstoskip)

    list_sorted_uniqwords = dict_sort(dict_uniqwords)

    ###print("Remaining words after filtering: ",len(list_allwordslemmatized))
    #topcnt=10 
    #list_word_topcnt=getsomestats(dict_uniqwords,txtdoc,topcnt)
    #print("Top",topcnt,": ",list_word_topcnt)
    ##################################################################
    print("---------------------------------------------------")
    ##################################################################
    # Output:   list_sorted_uniqwords , dict_uniqwords, list_tokens
    #
    print(list_sorted_uniqwords[0:10])

---------------------------------------------------
[('besluit', 8), ('successierecht', 6), ('regering', 5), ('gunstregime', 5), ('overdracht', 5), ('formulier', 4), ('onderneming', 4), ('vennootschap', 4), ('wetboek', 4), ('vaststelling', 3)]
---------------------------------------------------
[('besluit', 4), ('tarief', 4), ('vertaling', 4), ('januar', 4), ('minister', 4), ('dienst', 3), ('sätzen', 3), ('wörter', 3), ('wijziging', 2), ('vaststelling', 2)]
---------------------------------------------------
[('gutes', 17), ('nicht', 9), ('auf', 9), ('ke/estgb', 7), ('kostenlose', 6), ('zurverfügungstellung', 6), ('vorteil', 6), ('minister', 5), ('goed', 4), ('vertaling', 4)]
---------------------------------------------------
[('onderwijs', 14), ('promotie', 13), ('afdeling', 9), ('wetenschap', 7), ('specialisatie', 6), ('besluit', 6), ('code', 5), ('type', 4), ('gebied', 3), ('decreet', 3)]
---------------------------------------------------
[('besluit', 9), ('minister', 6), ('overwe

In [5]:
#compare a token against a tokenslist
def token_compare(token_totest,list_tokens,min_score:int = 0.6):
    #return list of tokens with similarity >= min_score 
    list_tokens_toreturn=[]
    tot_similar_word=0
    istax=False
    for token in list_tokens:
        #print("token_compare",token)
        similar=token_totest.similarity(token)
        #print(token_totest, "<->", token, similar)
        if similar >= min_score:
            list_tokens_toreturn.append([token_totest,token,similar])
            print("Passed:",token_totest,token,similar)
            #positive for tax
            istax=True
            tot_similar_word+=similar
    return istax, tot_similar_word, list_tokens_toreturn

def createlistofkeywords(numberofdocumenttoscan:int = 50,similar_doc:int = 0,min_score:int = 0.6, list_testwords:list = ['belasting','tax','venootschapsbelasting']):
    #numberofdocumenttoscan=50
    list_keep_tax_words=[]
    list_keep_pointer_taxdocs=[]
    for n in range(numberofdocumenttoscan):
        #print("$$$$$$$$$$",n,numberofdocumenttoscan)
        txtdoc = df['cleantextnl'].values[n]
        list_wordstoskip=['artikel','document'] #add here words to discard
        dict_uniqwords,list_tokens=nlp_cleanandlemmatize(txtdoc,list_wordstoskip) #get tokens
        #list_sorted_uniqwords = dict_sort(dict_uniqwords) #sort
        
        #print(n,"--------------------------")
        #words to check for

        #tokenize and check
        tot_similar_doc=0
        istax_doc=False
        for word in list_testwords:
            #print("@@@@",word,list_testwords,list_tokens)
            token_word = nlp(word)
            istax, tot_similar_word, list_res_tokens=token_compare(token_word,list_tokens,min_score)
            if istax:
                for taxtoken in list_res_tokens:
                    #print("@@@",taxtoken)
                    if taxtoken[1].lemma_.lower() not in list_keep_tax_words:
                        list_keep_tax_words.append(taxtoken[1].lemma_.lower())
                    
                tot_similar_doc+=tot_similar_word
            istax_doc |= istax
        #print("@@#",tot_similar_doc,similar_doc)
        #print("$$",n)
        if istax_doc and (tot_similar_doc >= similar_doc):
            list_keep_pointer_taxdocs.append(n)
            #print("TOTAL for doc:",n,"  score:",tot_similar_doc)
            #print(txtdoc)
    return list_keep_pointer_taxdocs , list_keep_tax_words




list_keep_tax_words=[]
#step1
list_keywords=['belasting','tax','fisc']
list_keep_pointer_taxdocs , list_keep_tax_words = createlistofkeywords(10,2,0.95,list_keywords)
print("step1:",list_keywords)
print("\n------------------------------------------------------------------------------------")
#step2
list_keywords+=list_keep_tax_words
list_keep_pointer_taxdocs , list_keep_tax_words = createlistofkeywords(20,4,0.90,list_keywords)
print("step2:",list_keywords)
print("\n------------------------------------------------------------------------------------")

#step3
list_keywords+=list_keep_tax_words
list_keep_pointer_taxdocs , list_keep_tax_words = createlistofkeywords(40,8,0.85,list_keywords)
print("step3:",list_keywords)
print("\n------------------------------------------------------------------------------------")

#step4
list_keywords+=list_keep_tax_words
list_keep_pointer_taxdocs , list_keep_tax_words = createlistofkeywords(80,16,0.80,list_keywords)
print("step4:",list_keywords)
print("\n------------------------------------------------------------------------------------")


#final
list_keywords+=list_keep_tax_words
print(list_keywords,"\n  taxdocus are: ",list_keep_pointer_taxdocs)
print("step5:",list_keywords)
print("\n------------------------------------------------------------------------------------")

df_keywords=pd.DataFrame(list_keywords,columns=['keywords'])
#save complete list of keywords
df_keywords.to_pickle("../data/tax_keywords.pkl")  


C:\Users\bmadmin\AppData\Local\Temp\ipykernel_6240\1237486297.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similar=token_totest.similarity(token)


Passed: belasting belasting 1.0
Passed: belasting belastingvermindering 0.9999999775566392
step1: ['belasting', 'tax', 'fisc']

------------------------------------------------------------------------------------
Passed: belasting successierechten 0.9043948372239012
Passed: belasting successierechten 0.9043948372239012
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belastingvermindering 0.9999999775566392
Passed: belasting belastingvermindering 0.9999999775566392
Passed: belasting belastingvermindering 0.9999999775566392
Passed: belasting belastingvermindering 0.9999999775566392
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
step2: ['belasting', 'tax', 'fisc', 'belasting', 'belas_tingvermindering']

------------------------------------------------------------------------------------
Passed: belasting successi

In [9]:


df_keywords = pd.read_pickle("../data/tax_keywords.pkl") 
list_keywords=list(df_keywords['keywords'])

#run on all documents

#show texts that are categorized as tax document
#for p in list_keep_pointer_taxdocs:#
    #print("-------------------------------------------------------------------------------")
    #print( df['cleantextnl'].values[p] )
    #input()
#print("docu: ",n,"######################################################################################################")
####################################################################
# output: list_keep_pointer_taxdocs , list_keep_tax_words


In [10]:
print("start for all")
list_keep_pointer_taxdocs , list_keep_tax_words = createlistofkeywords(10,0,0.95,list_keywords)
print("\n  taxdocus are: ",list_keep_pointer_taxdocs)
print("end for all")
print("stepEND:",list_keywords)
print("\n------------------------------------------------------------------------------------")


start for all


C:\Users\bmadmin\AppData\Local\Temp\ipykernel_6240\1237486297.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similar=token_totest.similarity(token)


Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: belasting belasting 1.0
Passed: verkeersbelasting belasting 0.9999999775566392
Passed: belasting belasting 1.0
Passed: verkeersbelasting belasting 0.9999999775566392
Passed: belastingverhoging belasting 0.9999999775566392
Passed: belastingvermindering belasting 0.9999999775566392
Passed: successierecht inkomstenbelastingen 1.0000000828577218
Passed: successierecht inkomstenbelastingen 1.0000000828577218
Passed: inkomstenbelasting inkomstenbelastingen 1.0000000828577218
Passed: inkomstenbelastingen inkomstenbelastingen 1.0
Passed: inkomstenjaar inkomstenbelastingen 1.0000000828577218
Passed: personenbelasting inkomstenbelastingen 1.0000000828577218
Passed: successierecht inkomstenbelastingen 1.0000000828577218
Passed: inkomstenbelasting inkomstenbelastingen 1.0000000828577218
Passed: inkomstenbelastingen inkomstenbelastingen 1.0
Passed: inkomstenjaar inkomstenbelastingen 1.00000008285